In [2]:
# Import libs
import matplotlib.pyplot as plt
import numpy as np
import time

def xshow(n, x, title, xlabel, ylabel):
    ''' 描绘序列, 绘制棒棒糖图(Lollipop Chart)

    :n: 与x等长的自然数序列
    :x: 需要绘制的序列
    :title: 图标题
    :xlabel: x轴标签
    :ylabel: y轴标签
    
    '''
    # plot the chart
    fig, ax = plt.subplots(figsize=(15,4), dpi= 100)
    ax.vlines(x=n, ymin=0, ymax=x, alpha=0.7, linewidth=2)                #plot line
    ax.scatter(x=n, y=x, s=75, alpha=0.7)                                 #plot dots

    # Title, Lable, Ticks, and Ylim
    ax.set_title(title, fontdict={'size':20})      
    ax.set_ylabel(ylabel,fontdict={'size':20})                             
    plt.tick_params(labelsize=20)
    ax.set_xticks(n)                                                       
    ax.set_xlabel(xlabel,fontdict={'size':20})
    if(round(np.min(x),2)<0):
        ax.set_ylim(np.min(x)-0.5,np.max(x)+0.5)
    else:
        ax.set_ylim(0, np.max(x)+0.5)

    # Annotation
    for i,j in zip(n,x):
        ax.text(i, j+.1, s=round(j, 2), horizontalalignment= 'center', verticalalignment='bottom', fontsize=14)

    plt.show()
    return

基2-DIF-FFT

In [3]:
def Reverse(x, N):
    '''对输入二进制数输出指定位数的倒序数
    
    :x: 输入
    :N: 指定总位数
    :y: 输出
    '''
    # 异常捕获
    try:
        if (x >= 2**N):
            raise ValueError("输入二进制数位数大于倒序位数")
    except Exception as e:
        print(e)
        return 0

    y = 0
    while (x >= 1):
        for i in range(0, N):
            y += x % 2 * 2**(N - 1 - i)  # 二进制表示下，y与x的权重正好是相反的故有“3-i”
            x = x // 2
    return y


def ButterflyCaculate(x, l, h):
    '''对给定向量x[l,...,h]做蝶形运算
    
    :x: 输入向量
    :l: 输入向量起点
    :h: 输入向量终点
    :y: 输出向量
    '''
    N = h - l + 1  # 向量长度
    if (N > 2):
        x = ButterflyCaculate(x, l, l + N // 2 - 1)  # 对前半部分序列做蝶形运算
        x = ButterflyCaculate(x, l + N // 2, h)  # 对后半部分序列做蝶形运算

    # 分两组进行两组之间的蝶形运算
    i = l
    while (i < l + N // 2):
        x[i], x[i + N // 2] = ButterflyCaculate2(x[i], x[i + N // 2], N, i - l)
        i = i + 1

    return x


def ButterflyCaculate2(x1, x2, N, k):
    '''对两点做蝶形运算y1=x1 + W_N^k * x2, y2=x1 - W_N^k * x2
    
    :x1: 输入点值1
    :x2: 输入点值2
    :N: 当前在做蝶形运算的点数
    :k: W上标k
    :return y1, y2: 输出点值
    '''
    theta = -2 * np.pi * k / N
    W = complex(np.cos(theta), np.sin(theta))
    x2 = x2 * W  # 一次复乘
    y1 = x1 + x2  # 第一次复加
    y2 = x1 - x2  # 第二次复加
    return y1, y2


def base2_DIF_FFT16(x):
    '''基2-DIF-16点FFT
    
    :x: 输入复数序列
    :return Xk: 输出复数序列
    '''
    # 异常捕获
    try:
        if (np.size(x) > 16):
            raise ValueError('!输入序列过长!')
    except Exception as e:
        print(e)
        return

    # 输入序列不足16点时，末端补零
    while (np.size(x) < 16):
        x = np.append(x, 0)

    # 序列倒序重排
    Rex = np.arange(0, 16, dtype=complex)
    for i in range(0, 16):
        Rex[i] = x[Reverse(i, 4)]

    # 对重排序列进行递归蝶形运算
    Xk = ButterflyCaculate(Rex, 0, 15)

    return Xk


基4-DIF-16点FFT

In [4]:
def Mutiplyj(x):
    '''复数乘j的简化运算
    
    :x: 输入值
    :return: 输出乘j后的结果
    '''
    return complex(-np.imag(x), np.real(x))


def ButterflyCaculate4(x0, x1, x2, x3, N, k):
    '''对四点做蝶形运算
    
    :x0: 输入点值1
    :x1: 输入点值2
    :x2: 输入点值3
    :x3: 输入点值4
    :N: 当前在做蝶形运算的点数
    :k: W上标k
    :return y0, y1, y2, y3, y4:输出点值
    '''
    theta = -2 * np.pi * k / N
    W1 = complex(np.cos(2 * theta), np.sin(2 * theta))
    W2 = complex(np.cos(theta), np.sin(theta))
    W3 = complex(np.cos(3 * theta), np.sin(3 * theta))
    x1 = x1 * W1
    x2 = x2 * W2
    x3 = x3 * W3
    U0 = x0 + x1
    U1 = x0 - x1
    U2 = x2 + x3
    U3 = -Mutiplyj(x2 - x3)
    y0 = U0 + U2
    y1 = U1 + U3
    y2 = U0 - U2
    y3 = U1 - U3
    return y0, y1, y2, y3


def ButterflyCaculateBase4(x, l, h):
    '''对给定向量x[l,...,h]做基4蝶形运算
    
    :x: 输入向量
    :l: 输入向量起点
    :h: 输入向量终点
    :y: 输出向量
    '''
    N = h - l + 1  # 向量长度
    if (N > 4):
        x = ButterflyCaculateBase4(x, l, l + N // 4 - 1)
        x = ButterflyCaculateBase4(x, l + N // 4, l + N // 2 - 1)
        x = ButterflyCaculateBase4(x, l + N // 2, l + 3 * N // 4 - 1)
        x = ButterflyCaculateBase4(x, l + 3 * N // 4, h)

    # 分四组进行四组之间的蝶形运算
    i = l
    while (i < l + N // 4):
        x[i], x[i +
                N // 4], x[i + N // 2], x[i + 3 * N // 4] = ButterflyCaculate4(
                    x[i], x[i + N // 4], x[i + N // 2], x[i + 3 * N // 4], N,
                    i - l)
        i = i + 1

    return x


def base4_DIF_FFT16(x):
    '''基4-DIF-16点FFT
    
    :x: 输入序列
    :return Xk: 输出序列
    '''
    # 异常捕获
    try:
        if (np.size(x) > 16):
            raise ValueError('!输入序列过长!')
    except Exception as e:
        print(e)
        return

    # 输入序列不足16点时，末端补零
    while (np.size(x) < 16):
        x = np.append(x, 0)

    # 序列倒序重排
    Rex = np.arange(0, 16, dtype=complex)
    for i in range(0, 16):
        Rex[i] = x[Reverse(i, 4)]

    # 对重排序列进行递归蝶形运算
    Xk = ButterflyCaculateBase4(Rex, 0, 15)

    return Xk




In [5]:


input2 = [4, -1, 2, 3, -3, -2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
input = [4, -1, 2, 3, -3, -2, 0, 1]
x = np.asarray(input2, dtype=complex)
output = base4_DIF_FFT16(x)
output2 = np.fft.fft(input2)
print(output)
print(output2)


[ 4.        +0.j          5.47987166+0.66190691j  6.29289322-4.12132034j
 -2.79897809-4.03153013j -1.        +7.j          7.97055096+4.79689699j
  7.70710678-0.12132034j  5.34855547-2.50966597j  2.        +0.j
  5.34855547+2.50966597j  7.70710678+0.12132034j  7.97055096-4.79689699j
 -1.        -7.j         -2.79897809+4.03153013j  6.29289322+4.12132034j
  5.47987166-0.66190691j]
[ 4.        +0.j          5.47987166+0.66190691j  6.29289322-4.12132034j
 -2.79897809-4.03153013j -1.        +7.j          7.97055096+4.79689699j
  7.70710678-0.12132034j  5.34855547-2.50966597j  2.        +0.j
  5.34855547+2.50966597j  7.70710678+0.12132034j  7.97055096-4.79689699j
 -1.        -7.j         -2.79897809+4.03153013j  6.29289322+4.12132034j
  5.47987166-0.66190691j]


In [6]:
def xDft(x, length):
    '''对序列做DFT变换

    :x: 待变换的序列
    :length: 序列长度
    :return Xf: 变换后序列

    '''
    Xf=np.zeros(length, dtype=complex)
    n=np.arange(0,length)
    for k in n:
        for m in n:
            Xf[k]+=x[m]*np.e**(-2*np.pi*1j*m*k/length)
    return Xf


In [8]:
# input = [3,2,1,0,1,0,0,6,5,8]
# input2 = [3,2,1,0,1,0,0,6,5,8,0,0,0,0,0,0]
# x = np.asarray(input2,dtype=complex)
x = np.random.rand(100,1)
n = np.arange(0,16)
# xshow(n,input2,'$x(n)$','$n$','$x[n]$')
start_time = time.time()
for i in range(0,100):
    Xkbase4 = base4_DIF_FFT16(x)
end_time = time.time()
time_base4 = (end_time-start_time)/100
# xshow(n,np.abs(Xkbase4),"Abs{Base4-FFT($x[n]$)}",'$k$','$X[k]$')
# xshow(n,np.real(Xkbase4),"Real{Base4-FFT($x[n]$)}",'$k$','$X[k]$')
# xshow(n,np.imag(Xkbase4),"Imag{Base4-FFT($x[n]$)}",'$k$','$X[k]$')
start_time = time.time()
for i in range(0,100):
    Xkbase2 = base2_DIF_FFT16(x)
end_time = time.time()
time_base2 = (end_time - start_time)/100
# xshow(n,np.abs(Xkbase2),"Abs{Base2-FFT($x[n]$)}",'$k$','$X[k]$')
# xshow(n,np.real(Xkbase2),"Real{Base2-FFT($x[n]$)}",'$k$','$X[k]$')
# xshow(n,np.imag(Xkbase2),"Imag{Base2-FFT($x[n]$)}",'$k$','$X[k]$')

start_time = time.time()
for i in range(0,100):
    y = xDft(x,16)
end_time = time.time()
time_dft = (end_time - start_time)/100

str = 'base4-FFT spend %f .' %(time_base4)
print(str)
str = 'base2-FFT spend %f .' %(time_base2)
print(str)
str = 'DFT spend %f .' %(time_dft)
print(str)
# xshow(n,np.abs(y),'abs{DFT(x[n])}','','X[k]')

!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!输入序列过长!
!